In [129]:
import pandas as pd

# q1

Compute the monthly payment ($B$) on a 30-year level payment mortgage assuming an annual mortgage rate of $5\%$ and an initial mortgage principal of $\$400,000$.




In [1]:
def get_B(c, n, M_0):
    """ Given some coupon rate (c), 
        number of periods in months(n), 
        and initial Mortgage principal (M_0),
        return the fixed monthly mortgage payment (B)"""
    c /= 12
    return (c*(1+c)**n*M_0) / ((1+c)**n - 1)

In [2]:
B = get_B(c=.05, n=30*12, M_0=400_000)

In [10]:
print(round(B, 2))

2147.29


In [74]:
def _cpr(t):
    """given some month t, 
    return the proper CPR (conditional prepayment rate) given the PSA benchmark"""
    if t <= 30:
        return .06 * (t/30)
    return .06

def smm(t, psa=100):
    """ given some month t, returnt he single month mortality rate"""
    psa /= 100
    return 1-(1-_cpr(t)*psa)**(1/12)

def calc_payments(m_0, c, pt, n, s, psa=100):
    """ m_0: initial mortgage balance
        c: mortgage rate
        pt: pass through rate
        n: term of the loan (months)
        s: seasoning (months)
        psa: prepayment multiplier
    """
    cols = ["smm", 
            "begin bal", 
            "monthly", 
            "interest paid", 
            "principal paid",
            "interest pass"
            "prepayment",
            "ending bal",
           ]
    df = pd.DataFrame([], columns=cols)
    
    prev_m = m_0
    for k in range(s + 1, n + 1):
        _smm = smm(k, psa=psa)
        b_k = prev_m*((c/12)/(1-(1+c/12)**-(n - s - k + 1)))
        i_k = prev_m*(c/12)
        p_k = b_k - i_k
        passthru = prev_m*pt/12
        prep_k = (prev_m - p_k)*_smm
        # print(f"monthly payment: {b_k}, interest payment: {i_k}, principal payment: {p_k}, prepayment: {prep_k}, ending_bal: {prev_m - prep_k - p_k}")

        new_bal = prev_m - prep_k - p_k

        to_append = {
            "smm": str(round(_smm*100,4))+"%", 
            "begin bal": prev_m, 
            "monthly": b_k, 
            "interest paid": i_k, 
            "principal paid": p_k,
            "interest pass": passthru,
            "prepayment": prep_k,
            "total p": p_k + prep_k,
            "ending bal": new_bal,
        }
        prev_m = new_bal
        df = df.append(to_append, ignore_index=True)

    return df

# q2

Consider a $\$400$ million pass-through MBS that has just been created (so the 'seasoning' of the pass-through is equal to $0$). The underlying pool of mortgages each has a maturity of 20 years and an annual mortgage coupon rate of $6\%$. The pass-through rate of the mortgage pool is $5\%$. Assuming a prepayment multiplier of $100$ PSA what is the total amount of interest paid to the pass-through investors?

$M_k = M_0\frac{(1+c)^n-(1+c)^k}{(1+c)^n - 1}$

$I_k := cM_{k-1}$

In [84]:
pass_through = .05
c = .06
M_0 = 400_000_000
n = 20*12

In [16]:
def get_remaining_principal(c, n, k, M_0):
    """ Given some yearly coupon rate with monthly compounding (c),
        number of periods in months (n),
        current period in months (k),
        and some initial principal (M_0)
        return a list of remaining principle from kth period
    """
    c /= 12
    return M_0* ((1+c)**n - (1+c)**k) / ((1+c)**n - 1)

# all_Ik = [get_remaining_principal(c=c, n=n, k=k, M_0=M_0)*c/12 for k in range(periods)]

In [128]:
mort = calc_payments(m_0=M_0, c=c, pt=pass_through, n=n, s=0, psa=100)
round(mort['interest pass'].sum()/1_000_000, 2)

171.18

# q3

 Referring to the same mortgage pass-through of the previous question, what is the total amount of the prepayments?

In [127]:
mort = calc_payments(m_0=M_0, c=c, pt=pass_through, n=n, s=0, psa=100)
print(f"total prepayment: {round(mort['prepayment'].sum()/1_000_000, 2)}")

total prepayment: 181.09


# q4

Referring to the same mortgage pass-through of the previous question, what is the total amount of the prepayments if the rate of prepayments increases to 200 PSA?

In [126]:
mort = calc_payments(m_0=M_0, c=c, pt=pass_through, n=n, s=0, psa=200)
print(f"total prepayment: {round(mort['prepayment'].sum()/1_000_000, 2)}")

total prepayment: 268.15


# q5

Suppose we construct principal-only (PO) and interest-only (IO) mortgage-backed securities (MBS) using the mortgage pass-through of the previous questions. Assume a prepayment multiplier of 100 PSA. What is the present value of the PO MBS if we use an annual risk-free rate of 4.5% to value the cash-flows?

A: 280.1

In [124]:
def p0_value(r, n, M_0, c):
    """ calculate the present value of principal
        Assuming no prepayment
    """
    r /= 12
    c /= 12
    frac1 = (c*M_0) / ((1+c)**n - 1)
    frac2 = ((1+r)**n - (1+c)**n)/((r-c)*(1+r)**n)
    return frac1 * frac2

In [98]:
mort = calc_payments(m_0=M_0, c=c, pt=pass_through, n=n, s=0, psa=100)
r = 0.045
princ = list(mort['total p'])

V_0 = 0
for i, p in enumerate(princ):
    V_0 += p/(1+r/12)**(i+1)
    
print(round(V_0/1_000_000, 2))


280.1


# q6

Referring to the previous question, what is the value of the IO MBS?

A: 133.23

In [ ]:
def F_0(c, r, n, M_0):
    c /= 12
    r /= 12
    frac1 = (c*(1+c)**n * M_0) / ((1+c)**n - 1)
    frac2 = ((1+r)**n - 1) / (r*(1+r)**n)
    return frac1 * frac2

In [146]:
mort = calc_payments(m_0=M_0, c=c, pt=pass_through, n=n, s=0, psa=100)
r = 0.045
princ = list(mort['interest pass'])

W_0 = 0
for i, p in enumerate(princ):
    i_ = p/(1+r/12)**(i+1)
    W_0 += i_
    
print(round(W_0/1_000_000, 2))

133.23


# q7

Referring to the previous question, what is the average life of the IO MBS?

In [152]:
mort = calc_payments(m_0=M_0, c=c, pt=pass_through, n=n, s=0, psa=100)
r = 0.045
ppaid = list(mort['interest pass'])

WAL_I = 0
tot = 0
for i, I_k in enumerate(ppaid):
    WAL_I += I_k*(i+1)    # weighted average life, NOT duration
    tot += I_k
WAL_I /= (12*tot)
print(round(WAL_I, 2))

6.01


# q8

Suppose now that you purchased the IO MBS of the previous question and that the price you paid was the same price that you calculated in the previous question. The risk-free interest rate suddenly changes from 4.5% to 3.5%. Everything else stays the same. How much money have you made or lost on your investment?

A: 7.17

In [122]:
mort = calc_payments(m_0=M_0, c=c, pt=pass_through, n=n, s=0, psa=100)
r = 0.035
princ = list(mort['interest pass'])

W_0_8 = 0
for i, p in enumerate(princ):
    i_ = p/(1+r/12)**(i+1)
    W_0_8 += i_
    
print(round((W_0_8-W_0)/1_000_000, 2))

7.17


# q9

Referring to the previous question, suppose the risk-free interest rate suddenly changes from 4.5% to 3.5% and that the pre-payment multiplier changes from 100 PSA to 150 PSA. How much money have you made or lost on your investment in the IO MBS?

A: -9.58

In [123]:
mort = calc_payments(m_0=M_0, c=c, pt=pass_through, n=n, s=0, psa=150)
r = 0.035
princ = list(mort['interest pass'])

W_0_9 = 0
for i, p in enumerate(princ):
    i_ = p/(1+r/12)**(i+1)
    W_0_9 += i_
    
print(round((W_0_9-W_0)/1_000_000, 2))

-9.58
